In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time, datetime, smtplib, random
from email.message import EmailMessage
import re
import time
import pandas as pd


# path = "/usr/bin/chromedriver" 
path = r'C:\Users\USER\Desktop\chromedriver.exe'

ser = Service(path)
chromeOptions = Options()
chromeOptions.add_argument('--no-sandbox')
# chromeOptions.add_argument('--headless')

driver = webdriver.Chrome(service = ser,  options=chromeOptions)


In [50]:
import os
from datetime import datetime

In [51]:
# xpaths and other input data

login_url = 'https://app.qandi.com/login'
#login_url = "https://app.thomsondigital2021.com/login"

email = 'uniqtd@gmail.com'
email_xp = '//*[@id="mobile_num"]'
next_btn_xp = '//*[@id="mobile-input-btn"]'
custom_exam_url = 'https://app.qandi.com/exam_custom'
#custom_exam_url = 'https://app.thomsondigital2021.com/exam_custom'

# For first question
o1_xp_f = '//*[@id="question_section"]/div[2]/div[2]/div[1]/div/label'
o2_xp_f = '//*[@id="question_section"]/div[2]/div[2]/div[2]/div/label'
o3_xp_f = '//*[@id="question_section"]/div[2]/div[2]/div[3]/div/label'
o4_xp_f = '//*[@id="question_section"]/div[2]/div[2]/div[4]/div/label'
o_xps_f = [o1_xp_f, o2_xp_f, o3_xp_f, o4_xp_f]
save_and_next_xp_f = '//*[@id="question_section"]/div[3]/a[1]'

# For other questions
o1_xp_o = '//*[@id="question_section"]/div/div[2]/div[2]/div[1]/div/label'
o2_xp_o = '//*[@id="question_section"]/div/div[2]/div[2]/div[2]/div/label'
o3_xp_o = '//*[@id="question_section"]/div/div[2]/div[2]/div[3]/div/label'
o4_xp_o = '//*[@id="question_section"]/div/div[2]/div[2]/div[4]/div/label'
o_xps_o = [o1_xp_o, o2_xp_o, o3_xp_o, o4_xp_o]

# other xpaths
take_test_for_sel_topic_xp = '//*[@id="topic_custom_footer"]/button'
go_for_it_xp = '//*[@id="goto-exam-btn"]'
phy_xp = '//*[@id="Physics-tab"]'
chem_xp = '//*[@id="Chemistry-tab"]'
math_xp = '//*[@id="Mathematics-tab"]'
save_and_next_xp_o = '//*[@id="question_section"]/div/div[3]/a[1]'

sub_test_xp = '//*[@id="bt-modal-confirm_over"]'


In [52]:
# Login manually 

driver.get(login_url)

# set implicit wait for 2s
driver.implicitly_wait(2)

# fill in the email and click
driver.find_element(By.XPATH, email_xp).send_keys(email)
driver.find_element(By.XPATH, next_btn_xp).click()


In [5]:
# Enter otp and sign in step


In [53]:
# Load question_data
q_data_df = pd.read_csv('q_data.csv')
topic_df = pd.read_csv('Topic_ID.csv')

In [54]:
def get_chp_ids(sub_xp):
    
    driver.get(custom_exam_url)
    driver.find_elements(By.XPATH, sub_xp)[0].click()
    pg_src_chp_id = driver.page_source
    chp_ids = re.findall(r'expand_topic_(\d+)', pg_src_chp_id)
    chp_ids = [int(i) for i in chp_ids]
    return chp_ids


In [55]:
def get_topic_ids(chp_id):
    # get subject and topic_ids from chapter id (chp_id)
    topic_ids = q_data_df[q_data_df['chap_id']==chp_id].top_id.unique()
    Subject = topic_df[topic_df['Topic ID']==topic_ids[0]].Subject.values[0]
    # print(Subject)
    
    tab_xp = f'//*[@id="{Subject}-tab"]'
    tab_elm = driver.find_elements(By.XPATH, tab_xp)[0]
    driver.execute_script("arguments[0].click();", tab_elm)
    
    chp_xp = f'//*[@id="expand_topic_{chp_id}"]'
    chp_elm = driver.find_elements(By.XPATH, chp_xp)[0]
    driver.execute_script("arguments[0].click();", chp_elm)

    return topic_ids

In [56]:
def start_test_and_get_id_of_first_q(topic_id):
    # Select topic and start the test
    topic_xp = f'//*[@id="chpt_topic_{topic_id}"]'
    
    while True:
        try:
            topic_elm = driver.find_elements(By.XPATH, topic_xp)[0]
            driver.execute_script("arguments[0].click();", topic_elm)
            break
        except Exception as e:
            print(e)
            print("invalid topic id")
            return -1
                
    take_test_for_sel_topic_elm = driver.find_elements(By.XPATH, take_test_for_sel_topic_xp)[0]
    driver.execute_script("arguments[0].click();", take_test_for_sel_topic_elm)

    # Exam Starts now
    go_for_it_elm = driver.find_elements(By.XPATH, go_for_it_xp)[0]
    driver.execute_script("arguments[0].click();", go_for_it_elm)
    
    # Get current question id
    q_id_pg_src = driver.page_source
    curr_q_id = re.findall(r'id="current_question" value="(\d+)"', q_id_pg_src)[0]
    curr_q_id  = int(curr_q_id)
    return curr_q_id


In [57]:
def choose_an_option_and_record(curr_q_id, record_df):
    curr_q_data = q_data_df[q_data_df['q_id']==curr_q_id].iloc[0,:].tolist()
    Response_data = [curr_q_data[1], True]
    curr_q_record = curr_q_data + Response_data
    record_df.loc[curr_q_id,:] = curr_q_record
    return record_df
# choose_an_option_and_record(curr_q_id, record_df)

In [58]:
def do_first_questions(o_xp):
    o_xp_elm = driver.find_elements(By.XPATH, o_xp)[0]
    driver.execute_script("arguments[0].click();", o_xp_elm)
    
    save_and_next_elm = driver.find_elements(By.XPATH, save_and_next_xp_f)[0]
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    return 


In [59]:
def do_other_questions(o_xp):
    o_xp_elm = driver.find_elements(By.XPATH, o_xp)[0]
    driver.execute_script("arguments[0].click();", o_xp_elm)
    
    save_and_next_elm = driver.find_elements(By.XPATH, save_and_next_xp_o)[0]
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    return 


In [60]:
def make_csv(record_df):
    topic_id = record_df.iloc[0,3]
    Subject = topic_df[topic_df['Topic ID']==topic_id].Subject.values[0]
    Topic = topic_df[topic_df['Topic ID']==topic_id].Topic.values[0]
    
    remove_symbols = r'\/:*?"<>|'
    for symbol in remove_symbols:
        Topic = Topic.replace(symbol, ',')
    file_name = f'botTestOutput/{Subject[0]}_{Topic}.csv'
    
    driver.save_screenshot(os.path.join(os.getcwd(),'botTestOutput',f'{Subject[0]}_{Topic}.png'))

    
    record_df.to_csv(file_name, index=False)
    


In [61]:
# save screenshots with the given tag in exception_screenshots folder
def save_screenshot(tag):
    now = datetime.now() # current date and time
    date_time = now.strftime("%m %d %Y-%H %M %S")
    return driver.save_screenshot(os.path.join(os.getcwd(),'exception_screenshots',f'{tag}_{date_time}.png'))

In [62]:
def attempt_topic_test(topic_id):
    curr_q_id = start_test_and_get_id_of_first_q(topic_id)
    
    if curr_q_id == -1:
        return pd.DataFrame()
    
    columns = 'q_id corr_opt diff_lev top_id chap_id opt_chosen isResponseCorrect'.split(' ')
    record_df = pd.DataFrame(columns = columns)
    record_df = choose_an_option_and_record(curr_q_id, record_df)
    correct_option_ind = record_df.loc[curr_q_id, 'corr_opt']-1
    o_xp = o_xps_f[correct_option_ind]
    
    try:
        do_first_questions(o_xp)
    except:
        return
    
    count = 1

    while True:
        try:
            # Do the questions till all questions in the topic are exhausted
            time.sleep(1)
            q_id_pg_src = driver.page_source
            curr_q_id = re.findall(r'id="current_question" value="(\d+)"', q_id_pg_src)[0]
            curr_q_id =int(curr_q_id)
            
            record_df = choose_an_option_and_record(curr_q_id, record_df)
            correct_option_ind = record_df.loc[curr_q_id, 'corr_opt']-1
            o_xp = o_xps_o[correct_option_ind]
            do_other_questions(o_xp)
            
            end_exam_xp = '//*[@id="endExam"]'
            
            count += 1
            
            # this is used for debugging purposes
            end_test_early = False

            try:
                end_exam_elm = driver.find_elements(By.XPATH, end_exam_xp)[0]
                test_condition = end_exam_elm.value_of_css_property('display')
                
                
                if test_condition!='none' or end_test_early:
                    print('No more questions available.')
                    save_screenshot("ques_over")
                    break
            except Exception as e:
                save_screenshot("random_error")
            
        except:
            print('Break due to other reason')
            save_screenshot("random_error")
            break
    return record_df


In [63]:
# To get all the chp_ids we need any subject_tab_xp and call get_chp_ids function 
chp_ids = get_chp_ids(math_xp)

In [65]:
def go_to_custom_exam_url():
    driver.get(custom_exam_url)
    get_topic_ids(chp_id)

# Start the topic test one by one

driver.get(custom_exam_url)

i = 0
start = 40

for chp_id in chp_ids[start:]:
    print(f"currently on iteration : {start+i}")
    i += 1
    
    topic_ids = get_topic_ids(chp_id)
    # print(topic_ids)
    # topic_ids = list(topic_ids)
    for topic_id in topic_ids:
        #get_topic_ids(chp_id)
        record_df = attempt_topic_test(topic_id)

        if record_df.empty:
            go_to_custom_exam_url()
            continue
            

        print("test attempted sucessfully")
        
        # submit the test 
        sub_test_elm = driver.find_elements(By.XPATH, sub_test_xp)[0]
        driver.execute_script("arguments[0].click();", sub_test_elm)
                


        # Make csv for the current topic
        make_csv(record_df)

#         driver.get(custom_exam_url)
#         time.sleep(1)
#         get_topic_ids(chp_id) 
        go_to_custom_exam_url()
        
        
        print(f'Test completed for the topic id: {topic_id}')

print('Done')
record_df

currently on iteration : 40
Break due to other reason
test attempted sucessfully
Test completed for the topic id: 3722
No more questions available.
test attempted sucessfully
Test completed for the topic id: 3723
Break due to other reason
test attempted sucessfully
Test completed for the topic id: 3724
No more questions available.
test attempted sucessfully
Test completed for the topic id: 3725
No more questions available.
test attempted sucessfully
Test completed for the topic id: 3721
currently on iteration : 41
No more questions available.
test attempted sucessfully
Test completed for the topic id: 3739
Break due to other reason
test attempted sucessfully
Test completed for the topic id: 3740
currently on iteration : 42
No more questions available.
test attempted sucessfully
Test completed for the topic id: 3727
No more questions available.
test attempted sucessfully
Test completed for the topic id: 3726
currently on iteration : 43
No more questions available.
test attempted sucessf

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x01079943+2595139]
	Ordinal0 [0x0100C9F1+2148849]
	Ordinal0 [0x00F04528+1066280]
	Ordinal0 [0x00F00DB4+1052084]
	Ordinal0 [0x00EF63BD+1008573]
	Ordinal0 [0x00EF6F7C+1011580]
	Ordinal0 [0x00EF65CA+1009098]
	Ordinal0 [0x00EF5BC6+1006534]
	Ordinal0 [0x00EF4AD0+1002192]
	Ordinal0 [0x00EF4FAD+1003437]
	Ordinal0 [0x00F05C4A+1072202]
	Ordinal0 [0x00F5C19D+1425821]
	Ordinal0 [0x00F4B9EC+1358316]
	Ordinal0 [0x00F5BAF2+1424114]
	Ordinal0 [0x00F4B806+1357830]
	Ordinal0 [0x00F26086+1204358]
	Ordinal0 [0x00F26F96+1208214]
	GetHandleVerifier [0x0121B232+1658114]
	GetHandleVerifier [0x012D312C+2411516]
	GetHandleVerifier [0x0110F261+560433]
	GetHandleVerifier [0x0110E366+556598]
	Ordinal0 [0x0101286B+2173035]
	Ordinal0 [0x010175F8+2192888]
	Ordinal0 [0x010176E5+2193125]
	Ordinal0 [0x010211FC+2232828]
	BaseThreadInitThunk [0x774B3744+36]
	RtlSetCurrentTransact